In [194]:
# importing function from a different ipny 
from ipynb.fs.full.dataset import spam_test_train_set
from ipynb.fs.full.dataset import confusion
import pandas as pd
import collections
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
pd.options.display.max_columns = 50

In [200]:
[train_email, test_email] = spam_test_train_set()
emails = pd.concat([train_email, test_email])
print(len(emails))

2018-12-03 01:12:11,172   ERROR:mailparser.utils:273:      parse_received More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from anderson.doramail.com 207.12.141.100 by coliform.funmail.co.uk InterMail vK.4.04.00.00 658-589-341 license 893694jz34ml9wal1qv276h1o63n3u76 with ESMTP id <20034613789721.SXCH195.coliform@anderson.doramail.com> for <kruscit@enron.com>; Sun, 5 Aug 2001 20:34:59 -0600 Received: from Matthew 206.81.96.2 by anderson.doramail.com iPlanet Messaging Server 5.2 HotFix 1.14 built Mar 18 2003 with ESMTPA id <0OMZ008USAR04O@anderson.doramail.com> for kruscit@enron.com; Sun, 5 Aug 2001 23:34:59 -0300 IDT
2018-12-03 01:12:11,460   ERROR:mailparser.utils:273:      parse_received More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from anderson.doramail.com 207

In [201]:
def iter_spacy(strs, tokenizer):
    words = []
    for s in strs:
        tokens = tokenizer(s)
        counts = collections.Counter(tokens.text.lower())
        #for (i,d) in enumerate(counts):
        #    words.append(d)
        yield counts
    #unique = reduce(lambda l, x: l.append(x) or l if x not in l else l, words, [])
    #print(len(unique))

In [205]:
bows_spacy = pd.DataFrame(iter_spacy(emails.subject, tokenizer )).fillna(0).astype(int)
bows_spacy_train = bows_spacy[:1600]
bows_spacy_test = bows_spacy[1600:]
#y_train= train_email['isspam']
#bows_spacy_test = pd.DataFrame(iter_spacy(test_email.subject, tokenizer )).fillna(0).astype(int)
#y_test= test_email['isspam']

In [206]:
bows_spacy_train.head()

,,,,!,"""",#,$,%,&,',(,),*,+,",",-,.,/,0,1,2,3,4,5,6,...,集,電,震,頻,願,駪,高,黴,鼭,鿡,볳,볷,븶,빫,,,,！,５,６,９,＼,￢,񽺺,򴩱
0,0,0,3,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [207]:
bows_spacy_test.head()

,,,,!,"""",#,$,%,&,',(,),*,+,",",-,.,/,0,1,2,3,4,5,6,...,集,電,震,頻,願,駪,高,黴,鼭,鿡,볳,볷,븶,빫,,,,！,５,６,９,＼,￢,񽺺,򴩱
1600,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1601,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1602,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1603,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1604,0,0,9,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [214]:
C = [0.01, 0.1, 1, 10, 100]
accuracy =[]
prec = []
true_pos =[]
false_pos = []
false_neg =[]
true_neg = []
for i in range(5):
    logreg = LogisticRegression(C=C[i])

    # Fit the classifier to the training data
    logreg.fit(bows_spacy_train, y_train)

    # Predict the labels of the test set: y_pred
    y_pred =logreg.predict(bows_spacy_test)
    y_pred_train =logreg.predict(bows_spacy_train)
    true_pos.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[0])
    false_pos.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[1])
    false_neg.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[2])
    true_neg.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[3])
    accuracy.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[4])
    prec.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[5])

/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6

### Accuracy and Precision on training set

In [209]:
metrics = pd.DataFrame(columns = ['true_pos','false_pos','false_neg','true_neg','Accuracy', 'Precision'],\
                      index = ['C=0.01', 'C=0.1', 'C=1' ,'C=10','C=100'])

In [210]:
metrics.Accuracy = accuracy
metrics.Precision = prec
metrics.true_pos = true_pos
metrics.true_neg = true_neg
metrics.false_neg = false_neg
metrics.false_pos = false_pos

In [211]:
metrics.head()

,true_pos,false_pos,false_neg,true_neg,Accuracy,Precision
C=0.01,611.0,173.0,189.0,627.0,0.773750,0.76375
C=0.1,621.0,147.0,179.0,653.0,0.796250,0.77625
C=1,637.0,139.0,163.0,661.0,0.811250,0.79625
C=10,683.0,144.0,117.0,656.0,0.836875,0.85375
C=100,687.0,132.0,113.0,668.0,0.846875,0.85875


In [218]:
true_pos = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[0]
false_pos = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[1]
false_neg = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[2]
true_neg = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[3]
accuracy = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[4]
prec = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[5] 

/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/home/titli/anaconda3/lib/python3.6/site-packages/ipykern

In [221]:
metrics_test = pd.DataFrame(columns = ['true_pos','false_pos','false_neg','true_neg','Accuracy', 'Precision'],\
                           index = ['C=100'])
metrics_test.Accuracy = accuracy
metrics_test.Precision = prec
metrics_test.true_pos = true_pos
metrics_test.true_neg = true_neg
metrics_test.false_neg = false_neg
metrics_test.false_pos = false_pos
metrics_test.head()

,true_pos,false_pos,false_neg,true_neg,Accuracy,Precision
C=100,165.0,89.0,35.0,111.0,0.69,0.825
